In [3]:
pip install pywin32 PyPDF2 pdfplumber Pillow pymongo openpyxl fpdf


Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import win32com.client
import pdfplumber
from pymongo import MongoClient
from openpyxl import Workbook
from fpdf import FPDF
from PIL import Image
from io import BytesIO

# Constants
MONGODB_URI = "mongodb://localhost:27017/"
DB_NAME = "shirt_db"
COLLECTION_NAME = "shirt_data"
OUTLOOK_ACCOUNT = "Madeshkanna2004@outlook.com"  # Replace with your email
EMAIL_SUBJECT = "Processed Shirt Data"
OUTPUT_FOLDER = "output_files"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Step 1: Download the PDF from Outlook Mail
def download_pdf_from_outlook():
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.GetDefaultFolder(6)  # 6 refers to the Inbox folder
    messages = inbox.Items
    print(messages)
    messages = messages.Restrict("[ReceivedTime] >= '2024-01-01'")  # Optional filter for recent emails
    for message in messages:
        if "Shirt Data" in message.Subject:  # Adjust subject filter as needed
            if message.Attachments.Count > 0:
                for attachment in message.Attachments:
                    if attachment.FileName.endswith(".pdf"):
                        file_path = os.path.join(OUTPUT_FOLDER, attachment.FileName)
                        attachment.SaveAsFile(file_path)
                        print(f"PDF downloaded to: {file_path}")
                        return file_path
            else:
                print(f"No attachments found in email: {message.Subject}")
    raise FileNotFoundError("PDF not found in Outlook emails.")

# Step 2: Parse the PDF
def parse_pdf(file_path):
    data = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                data.extend(table)
            # Save images if any (assumes images are inline with content)
            for img in page.images:
                img_path = os.path.join(OUTPUT_FOLDER, f"image_{img['x0']}.png")
                with open(img_path, "wb") as f:
                    f.write(img["stream"])
    return data

# Step 3: Store in MongoDB
def store_in_mongodb(data):
    client = MongoClient(MONGODB_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    collection.insert_many(data)
    client.close()

# Step 4: Retrieve and Store in Excel
def retrieve_and_store_in_excel():
    client = MongoClient(MONGODB_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    data = list(collection.find({}, {"_id": 0}))
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Shirt Data"
    headers = list(data[0].keys()) + ["Item Price", "Total Price"]
    ws.append(headers)
    
    grand_total = 0
    for row in data:
        row_data = list(row.values())
        item_price = 10  # Example price; replace with actual logic
        total_price = item_price * int(row.get("quantity", 1))
        grand_total += total_price
        ws.append(row_data + [item_price, total_price])
    
    ws.append([])
    ws.append(["Grand Total", grand_total])
    excel_path = os.path.join(OUTPUT_FOLDER, "shirt_data.xlsx")
    wb.save(excel_path)
    client.close()
    return excel_path

# Step 5: Generate PDF Report
def generate_pdf_report(data):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    
    for record in data:
        image_path = record.pop("image_path", None)  # Assume "image_path" is in data
        pdf.cell(200, 10, txt="Shirt Features:", ln=True, align="L")
        for key, value in record.items():
            pdf.cell(200, 10, txt=f"{key}: {value}", ln=True, align="L")
        if image_path and os.path.exists(image_path):
            pdf.image(image_path, x=150, y=pdf.get_y(), w=40)
            pdf.ln(60)
        else:
            pdf.ln(10)
    
    table_start_y = pdf.get_y() + 10
    pdf.set_xy(10, table_start_y)
    pdf.cell(200, 10, txt="Shirt Table:", ln=True, align="L")
    for row in data:
        pdf.cell(200, 10, txt=" | ".join(map(str, row.values())), ln=True, align="L")
    
    pdf_path = os.path.join(OUTPUT_FOLDER, "shirt_report.pdf")
    pdf.output(pdf_path)
    return pdf_path

# Step 6: Send Email with Attachments
def send_email_with_attachments(excel_path, pdf_path):
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    mail.To = "Madeshkanna2004@outlook.com"  # Replace with recipient email
    mail.Subject = EMAIL_SUBJECT
    mail.Body = "Please find the processed shirt data attached."
    mail.Attachments.Add(excel_path)
    mail.Attachments.Add(pdf_path)
    mail.Send()

# Main Workflow
try:
    pdf_path = download_pdf_from_outlook()
    parsed_data = parse_pdf(pdf_path)
    store_in_mongodb(parsed_data)
    excel_path = retrieve_and_store_in_excel()
    pdf_report_path = generate_pdf_report(parsed_data)
    send_email_with_attachments(excel_path, pdf_report_path)
    print("Process completed successfully.")
except Exception as e:
    print(f"Error: {e}")


<COMObject <unknown>>
Error: PDF not found in Outlook emails.
